In [1]:
### pakiety - potrzebne w colabie
# !pip install datasets
# !pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip install jiwer
# !pip install transformers[torch]
# !pip install accelerate==0.30.0

In [2]:
### login do huggingface - potrzebny w colabie
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "pl", split="train")
common_voice_test = load_dataset("mozilla-foundation/common_voice_17_0", "pl", split="test")
common_voice_eval = load_dataset("mozilla-foundation/common_voice_17_0", "pl", split="validation")

# ograniczenie wielkości datasetów do testów
#common_voice_train = common_voice_train.select(range(1024))
common_voice_test = common_voice_test.select(range(1024))

In [4]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_eval = common_voice_eval.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence,variant
0,Dlatego też ta debata jest nam potrzebna,
1,"""Zawrócił i wkrótce straciłem go z oczu.""",
2,"Jeśli chodzi konkretnie o sprawę współpracy administracyjnej, uważam, że automatyczna wymiana informacji jest korzystna",
3,"Po raz kolejny przekonaliśmy się, że siła Europy leży we wspólnych działaniach",
4,"Chiny zakupiły już obligacje greckie, hiszpańskie i portugalskie",
5,Bo krew na razie jeszcze mi się nie zmęczyła nigdy.,
6,Mógłbym wyobrazić sobie konieczność prowadzenia europejskiej zagranicznej polityki rolnej,
7,"Są dwa aspekty sprawozdania i rezolucji, które chciałbym podjąć",
8,"Jestem przekonana, że możliwe jest osiągnięcie kompromisu na podstawie państwa wstępnych sugestii",
9,"Zero tak napisane, że wygląda jak sześć.",


In [6]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\«\»\–\—\’\„\…]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [7]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)
common_voice_eval = common_voice_eval.map(remove_special_characters)

show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,variant
0,ale co z tobą się stanie,
1,jedynym regionem który nie został ujęty w procesie liberalizacji reżimu wizowego jest kosowo,
2,mimo wszystko ich stan nadal jest bardzo zły,
3,w imieniu nas wszystkich nalegam na pozytywną odpowiedź w tej sprawie,
4,prawda jest taka że sytuacja jest niezwykle delikatna,
5,przynoszą one skutek o czym wiem z cotygodniowych sprawozdań,
6,w naszym społeczeństwie bezpieczeństwo zależy od zmian,
7,tak się składa że dla nas nie oznacza to wprowadzenia podatku finansowego,
8,dla zapewnienia stabilności finansowej decydujące znaczenie mają,
9,do końca czyli do chwili złożenia ostatecznej wersji sprawozdania nad którą obecnie pracujemy,


In [8]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[ą]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[ę]', 'e', batch["sentence"])
    batch["sentence"] = re.sub('[ó]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[ł]', 'l', batch["sentence"])
    batch["sentence"] = re.sub('[ż]', 'rz', batch["sentence"])
    batch["sentence"] = re.sub('[ń]', 'ni', batch["sentence"])
    batch["sentence"] = re.sub('[ć]', 'ci', batch["sentence"])
    batch["sentence"] = re.sub('[ś]', 'si', batch["sentence"])
    batch["sentence"] = re.sub('[ź]', 'zi', batch["sentence"])
    batch["sentence"] = re.sub('[ü]', 'u', batch["sentence"])
    return batch

common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)
common_voice_eval = common_voice_eval.map(replace_hatted_characters)

Map:   0%|          | 0/9230 [00:00<?, ? examples/s]

In [9]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_eval = common_voice_eval.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_eval.column_names)

Map:   0%|          | 0/20729 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Map:   0%|          | 0/9230 [00:00<?, ? examples/s]

In [10]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [11]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [12]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [13]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [14]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [15]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [16]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [17]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [18]:
common_voice_train[0]["path"]

'/home/mati/.cache/huggingface/datasets/downloads/extracted/ce100de2fe6aaf90a46e818cb55859d23995530367b5864185bd5a28ddb13e88/pl_train_0/common_voice_pl_21655246.mp3'

In [19]:
common_voice_train[0]["audio"]

{'path': '/home/mati/.cache/huggingface/datasets/downloads/extracted/ce100de2fe6aaf90a46e818cb55859d23995530367b5864185bd5a28ddb13e88/pl_train_0/common_voice_pl_21655246.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.05379274e-06, -3.55337511e-06, -4.30422733e-06]),
 'sampling_rate': 48000}

In [20]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_eval = common_voice_eval.cast_column("audio", Audio(sampling_rate=16_000))

In [21]:
common_voice_train[0]["audio"]

{'path': '/home/mati/.cache/huggingface/datasets/downloads/extracted/ce100de2fe6aaf90a46e818cb55859d23995530367b5864185bd5a28ddb13e88/pl_train_0/common_voice_pl_21655246.mp3',
 'array': array([-2.96461532e-21,  3.17637355e-21, -2.11758237e-20, ...,
         7.11472239e-07, -3.45709395e-07, -2.75473758e-06]),
 'sampling_rate': 16000}

In [22]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

ale jutro bedzie nam jurz lepiej jutro staniemy sie innymi ludzimi nieprawdarz kapralu


In [23]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: unia europejska opiera sie na relacji solidarnosici miedzy panistwami czlonkowskimi
Input array shape: (81216,)
Sampling rate: 16000


In [24]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [25]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)
common_voice_eval = common_voice_eval.map(prepare_dataset, remove_columns=common_voice_eval.column_names)

Map:   0%|          | 0/9230 [00:00<?, ? examples/s]

/home/mati/repo/ZZSN_projekt/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [26]:
print(common_voice_train.column_names)

['input_values', 'input_length', 'labels']


In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [28]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [29]:
wer_metric = load_metric("wer")

/tmp/ipykernel_1251/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [30]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [31]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    #"facebook/wav2vec2-base-960h", 
    "./models/wav2vec",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True
)

In [32]:
def change_grad(module, on):
    for param in module.parameters():
        param.requires_grad=on 
    module.requires_grad=on

In [33]:
model.freeze_feature_encoder()

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="out",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=4,
  eval_accumulation_steps=4, 
  eval_strategy="steps",
  num_train_epochs=30, # powinno być około 30 
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=4,
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [38]:
# Function to evaluate WER
def evaluate(model, processor, dataset):
    model.eval()
    predictions = []
    references = []
    
    for batch in dataset:
        input_dict = processor(batch["input_values"], return_tensors="pt", padding=True, sampling_rate=16000)
        with torch.no_grad():
            logits = model(input_dict.input_values.to("cuda")).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.decode(predicted_ids[0])
        predictions.append(transcription)
        references.append(processor.decode(batch["labels"]))

    wer_score = wer_metric.compute(predictions=predictions, references=references)
    return wer_score, predictions, references

In [39]:
wer_score, predictions, references = evaluate(model, processor, common_voice_eval)

print(f"Word Error Rate (WER): {wer_score}")
# Optionally, print some predictions and references for manual inspection
for ref, pred in zip(references[:10], predictions[:10]):
    print(f"Reference: {ref}")
    print(f"Prediction: {pred}")
    print("------")

Word Error Rate (WER): 0.2837791267926729
Reference: usiadl na szyszce i wsluchal sie
Prediction: usiad na szyszce i sluchal sie
------
Reference: poszedl do domu
Prediction: poszedl to domu
------
Reference: wydawalo mu sie rze jest martwy
Prediction: wydawalo mu sie rze jest martwy
------
Reference: inrzynier nie rozumial
Prediction: i rzynier nie rozumial
------
Reference: wspina sie na gure skad morzna spasici i zabici sie
Prediction: spira sie na gure skad morzna sparazici zabici sie
------
Reference: na nim skupilasi swa uwage gdy wchodzilem
Prediction: na zim skupilesi slom uwage gdy wchodzilem
------
Reference: dokonicze jeden poziom i ide spaci
Prediction: dokoniczy jeden plozia mide spaci
------
Reference: to jest nieokresilone ale pewne
Prediction: to jest nieokresilone ale w pewne
------
Reference: krul odbywa dalekie wycieczki lodzia
Prediction: krul odbywa dalekie wycieczki lodzia
------
Reference: posiliznalesi sie wiec wolniej ostrorzniej
Prediction: pusili znalasi sie 